In [1]:
import os
import numpy as np
import pickle
import glob
import networkx as nx
from tqdm import tqdm
from deepcrime_scripts.utils import *
import statsmodels.stats.proportion as smp

HOME_DIR = '/home/DC_replication'
OUTPUT_DIR = './output'

In [2]:
target_subject = 'audio'
total_model_num = 20

In [4]:
# def remove_dup(path):
#     lines = []
#     with open(path, 'r') as f:
#         for line in f:
#             if line not in lines:
#                 lines.append(line)
#     with open(path, 'w') as f:        
#         for line in lines:
#             f.write(line)

def read_mutant_accs_file(path):
    # remove_dup(path)
    mutants_killed = {}
    with open(path, 'r') as f:
        for line in f:
            op_no, mut_no, model_no, acc, killed = line.strip().split(',')
            model_no = int(model_no)

            if model_no != 0:
                continue

            if op_no not in mutants_killed:
                mutants_killed[op_no] = {'killed': 0, 'total': 0}

            if killed == 'True':
                mutants_killed[op_no]['killed'] += 1
            mutants_killed[op_no]['total'] += 1

    return mutants_killed

path_killed_by_train = f'{OUTPUT_DIR}/mutants_accs_trainset.csv'
path_killed_by_strong_test = f'{OUTPUT_DIR}/mutants_accs_strong_testset.csv'
path_killed_by_weak_test = f'{OUTPUT_DIR}/mutants_accs_weak_testset.csv'

m_killed_train = read_mutant_accs_file(path_killed_by_train)
m_killed_strong_test = read_mutant_accs_file(path_killed_by_strong_test)
m_killed_weak_test = read_mutant_accs_file(path_killed_by_weak_test)

In [10]:
for mut_op, train_result in m_killed_train.items():
    if train_result['killed'] == 0:
        continue    
    num_killed_by_train = train_result['killed']    
    num_killed_by_strong_test = m_killed_strong_test[mut_op]['killed']    
    num_killed_by_weak_test = m_killed_weak_test[mut_op]['killed']

    ms_weak = num_killed_by_weak_test / num_killed_by_train
    ms_strong = num_killed_by_strong_test / num_killed_by_train    

    if num_killed_by_train < 10:
        sensitivity = -1
    else:
        sensitivity = round((ms_strong - ms_weak) * 100 / ms_strong, 2)

    print(f'{mut_op}: {sensitivity}, {ms_strong}, {ms_weak}')
    print(f'{num_killed_by_train}, {num_killed_by_strong_test}, {num_killed_by_weak_test}')
    print('='*30)

    

1: 70.0, 1.0, 0.3
20, 20, 6
2: 68.42, 1.0, 0.3157894736842105
19, 19, 6
3: 20.0, 1.0, 0.8
20, 20, 16
4: 45.0, 1.0, 0.55
20, 20, 11
